In [8]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D

# Option: forcer CPU si besoin de debug GPU
tf.config.set_visible_devices([], 'GPU')

LONGUEUR_IMAGE = 28
LARGEUR_IMAGE = 28


In [9]:
# Assure-toi que les deux fichiers sont dans analyse_image/datas/
train_path = 'datas/fashion-mnist_train.csv'
test_path  = 'datas/fashion-mnist_test.csv'

observations_entrainement = pd.read_csv(train_path)
observations_test = pd.read_csv(test_path)

# X = pixels (colonnes 1..784), y = label (colonne 0)
X = np.array(observations_entrainement.iloc[:, 1:])
y = keras.utils.to_categorical(np.array(observations_entrainement.iloc[:, 0]), num_classes=10)

X_test = np.array(observations_test.iloc[:, 1:])
y_test = keras.utils.to_categorical(np.array(observations_test.iloc[:, 0]), num_classes=10)


In [10]:
X_apprentissage, X_validation, y_apprentissage, y_validation = train_test_split(
    X, y, test_size=0.2, random_state=13
)

def prep(x):
    x = x.reshape(x.shape[0], LARGEUR_IMAGE, LONGUEUR_IMAGE, 1).astype('float32')
    x /= 255.0
    return x

X_apprentissage = prep(X_apprentissage)
X_validation    = prep(X_validation)
X_test          = prep(X_test)

X_apprentissage.shape, X_validation.shape, X_test.shape


((48000, 28, 28, 1), (12000, 28, 28, 1), (10000, 28, 28, 1))

In [11]:
dimentionImage = (LARGEUR_IMAGE, LONGUEUR_IMAGE, 1)

model = Sequential([
    Input(shape=dimentionImage),
    Conv2D(32, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 5408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       692,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 693,962 (2.65 MB)

 Trainable params: 693,962 (2.65 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(
    X_apprentissage, y_apprentissage,
    batch_size=256,
    epochs=5,          # minimal viable
    verbose=1,
    validation_data=(X_validation, y_validation)
)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Erreur test: {test_loss:.4f}")
print(f"Précision test: {test_acc:.4f}")

Epoch 1/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8152 - loss: 0.5354 - val_accuracy: 0.8608 - val_loss: 0.3990
Epoch 2/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8754 - loss: 0.3566 - val_accuracy: 0.8867 - val_loss: 0.3267
Epoch 3/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8893 - loss: 0.3103 - val_accuracy: 0.8905 - val_loss: 0.3111
Epoch 4/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8965 - loss: 0.2899 - val_accuracy: 0.8923 - val_loss: 0.3056
Epoch 5/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9051 - loss: 0.2672 - val_accuracy: 0.9005 - val_loss: 0.2871
Erreur test: 0.2780
Précision test: 0.9015


In [13]:
# Dossier modele/ doit exister
import os
os.makedirs('modele', exist_ok=True)

# Sauvegarde architecture
model_json = model.to_json()
with open("modele/modele.json", "w") as f:
    f.write(model_json)

# Sauvegarde poids
model.save_weights("modele/modele.weights.h5")
print("Modèle sauvegardé dans modele/")

Modèle sauvegardé dans modele/
